pip install "mcp[cli]"

In [1]:
!ls

mcp-integration.ipynb  Previous-agent  README.md  requirement.txt


In [ ]:
# Base COde for MCP integration with Retell AI Voice Agents

import dotenv, os
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

dotenv.load_dotenv()  # Load environment variables from .env file


async def test_retell_mcp():
    # 1. Define the server parameters (matching your JSON config)
    server_params = StdioServerParameters(
        command="npx",
        args=["-y", "@abhaybabbar/retellai-mcp-server"],
        env={
            "RETELL_API_KEY": os.getenv("RETELL_API_KEY", "YOUR_RETELL_API_KEY_HERE"),
            "PATH": "/usr/local/bin:/usr/bin:/bin" # Ensure npx is in the path
        }
    )

    print("🚀 Connecting to Retell AI MCP Server...")

    # 2. Start the server and create a session
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # 3. List available tools (to see what's available)
            print("\n--- Available Tools ---")
            tools = await session.list_tools()
            for tool in tools.tools:
                print(f"Tool: {tool.name} - {tool.description}")

            # 4. Example: Access your Voice Agents
            # The tool name usually matches 'list_agents' or similar in this server
            print("\n--- Fetching Voice Agents ---")
            try:
                # We call the tool by name. Note: check the tool list output 
                # above to confirm the exact name (e.g., "list_agents")
                result = await session.call_tool("list_agents", arguments={})
                print("Result:", result.content[0].text)
            except Exception as e:
                print(f"❌ Error calling tool: {e}")

if __name__ == "__main__":
    asyncio.run(test_retell_mcp())

In [ ]:
# ✅ WORKING CODE - List all Retell AI MCP Tools
import dotenv, os
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

dotenv.load_dotenv()

# Get the full system PATH (including NVM for npx)
full_path = "/home/hex/.nvm/versions/node/v24.12.0/bin:/home/hex/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"

server_params = StdioServerParameters(
    command="npx",
    args=["-y", "@abhaybabbar/retellai-mcp-server"],
    env={
        "RETELL_API_KEY": os.getenv("RETELL_API_KEY"),
        "PATH": full_path
    }
)

print("🚀 Connecting to Retell AI MCP Server...\n")

# Use await directly in Jupyter (not asyncio.run)
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # List all available tools
        print("=" * 60)
        print("🛠️  AVAILABLE RETELL AI MCP TOOLS")
        print("=" * 60)
        
        tools = await session.list_tools()
        
        for i, tool in enumerate(tools.tools, 1):
            print(f"\n{i}. 📦 {tool.name}")
            print(f"   Description: {tool.description}")
            if hasattr(tool, 'inputSchema') and tool.inputSchema:
                print(f"   Input Schema: {tool.inputSchema}")
        
        print("\n" + "=" * 60)
        print(f"Total Tools: {len(tools.tools)}")
        print("=" * 60)

In [14]:
# 🔧 Alternative Approach: List Retell AI Capabilities via Direct API
import os
import requests
from dotenv import load_dotenv
import json

load_dotenv()

RETELL_API_KEY = os.getenv("RETELL_API_KEY")
BASE_URL = "https://api.retellai.com"

headers = {
    "Authorization": f"Bearer {RETELL_API_KEY}",
    "Content-Type": "application/json"
}

print("=" * 70)
print("🔍 RETELL AI - AVAILABLE API ENDPOINTS & TOOLS")
print("=" * 70)

# Define all available Retell AI endpoints
endpoints = {
    "Agents": [
        {"method": "GET", "path": "/list-agents", "description": "List all voice agents"},
        {"method": "POST", "path": "/create-agent", "description": "Create a new voice agent"},
        {"method": "GET", "path": "/get-agent/{agent_id}", "description": "Get agent details"},
        {"method": "PATCH", "path": "/update-agent/{agent_id}", "description": "Update agent configuration"},
        {"method": "DELETE", "path": "/delete-agent/{agent_id}", "description": "Delete an agent"},
    ],
    "Phone Calls": [
        {"method": "POST", "path": "/create-phone-call", "description": "Initiate outbound phone call"},
        {"method": "GET", "path": "/list-calls", "description": "List all phone calls"},
        {"method": "GET", "path": "/get-call/{call_id}", "description": "Get call details"},
    ],
    "Web Calls": [
        {"method": "POST", "path": "/create-web-call", "description": "Create web call access token"},
        {"method": "POST", "path": "/register-call", "description": "Register web call"},
    ],
    "Phone Numbers": [
        {"method": "GET", "path": "/list-phone-numbers", "description": "List purchased phone numbers"},
        {"method": "POST", "path": "/create-phone-number", "description": "Purchase new phone number"},
        {"method": "PATCH", "path": "/update-phone-number/{number}", "description": "Update phone number config"},
        {"method": "DELETE", "path": "/delete-phone-number/{number}", "description": "Release phone number"},
    ],
    "LLM & Prompts": [
        {"method": "GET", "path": "/list-llms", "description": "List available LLM models"},
        {"method": "POST", "path": "/create-custom-llm", "description": "Add custom LLM endpoint"},
    ],
    "Voice": [
        {"method": "GET", "path": "/list-voices", "description": "List available voices"},
    ],
    "Analytics": [
        {"method": "GET", "path": "/analytics", "description": "Get usage analytics"},
        {"method": "GET", "path": "/get-call-recording/{call_id}", "description": "Download call recording"},
    ]
}

# Display all available endpoints
for category, items in endpoints.items():
    print(f"\n📦 {category}")
    print("-" * 70)
    for item in items:
        print(f"   {item['method']:6} {item['path']:35} | {item['description']}")

print("\n" + "=" * 70)

# Test: List actual agents
print("\n🧪 TESTING: Fetching your actual voice agents...")
print("-" * 70)
try:
    response = requests.get(f"{BASE_URL}/list-agents", headers=headers)
    if response.status_code == 200:
        agents = response.json()
        print(f"✅ Found {len(agents)} agent(s):\n")
        for agent in agents:
            print(f"   🤖 Agent ID: {agent.get('agent_id', 'N/A')}")
            print(f"      Name: {agent.get('agent_name', 'Unnamed')}")
            print(f"      Voice: {agent.get('voice_id', 'N/A')}")
            print(f"      Language: {agent.get('language', 'N/A')}")
            print()
    else:
        print(f"❌ Error: {response.status_code} - {response.text}")
except Exception as e:
    print(f"❌ Error: {e}")

print("=" * 70)

🔍 RETELL AI - AVAILABLE API ENDPOINTS & TOOLS

📦 Agents
----------------------------------------------------------------------
   GET    /list-agents                        | List all voice agents
   POST   /create-agent                       | Create a new voice agent
   GET    /get-agent/{agent_id}               | Get agent details
   PATCH  /update-agent/{agent_id}            | Update agent configuration
   DELETE /delete-agent/{agent_id}            | Delete an agent

📦 Phone Calls
----------------------------------------------------------------------
   POST   /create-phone-call                  | Initiate outbound phone call
   GET    /list-calls                         | List all phone calls
   GET    /get-call/{call_id}                 | Get call details

📦 Web Calls
----------------------------------------------------------------------
   POST   /create-web-call                    | Create web call access token
   POST   /register-call                      | Register web call



---

## 🧪 Test Your MCP Server Setup

After reloading VS Code, try these commands in **GitHub Copilot Chat**:

### Quick Tests:
1. **Check MCP Connection**: `@workspace show me connected MCP servers`
2. **List Tools**: `@workspace what Retell AI tools are available?`
3. **List Agents**: `@workspace list all my Retell AI agents`
4. **Get Agent Details**: `@workspace show details for my first agent`
5. **Create Web Call**: `@workspace create a web call token for agent_594b1eb4983ebeb449c2e17c47`

### Expected Behavior:
- ✅ Copilot should connect to the Retell AI MCP server automatically
- ✅ You'll see responses with actual data from your Retell AI account
- ✅ All 13 of your voice agents should be accessible

---

In [ ]:
import dotenv, os
dotenv.load_dotenv()


os.getenv("RETELL_API_KEY", False)

In [1]:
import dotenv, os
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

dotenv.load_dotenv()  # Load environment variables from .env file


ImportError: cannot import name 'ClientSession' from partially initialized module 'mcp' (most likely due to a circular import) (/home/hex/Documents/devops/voice-agent/Retell-voice-agent/mcp.py)

In [6]:
server_params = StdioServerParameters(
        command="npx",
        args=["-y", "@abhaybabbar/retellai-mcp-server"],
        env={
            "RETELL_API_KEY": os.getenv("RETELL_API_KEY", "YOUR_RETELL_API_KEY_HERE"),
            # "PATH": "/usr/local/bin:/usr/bin:/bin" # Ensure npx is in the path
        }
    )


In [7]:
async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # 3. List available tools (to see what's available)
            print("\n--- Available Tools ---")
            tools = await session.list_tools()
            for tool in tools.tools:
                print(f"Tool: {tool.name} - {tool.description}")



--- Available Tools ---


  + Exception Group Traceback (most recent call last):
  |   File "/home/hex/Documents/devops/voice-agent/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_158124/1663021332.py", line 1, in <module>
  |     async with stdio_client(server_params) as (read, write):
  |   File "/usr/lib/python3.12/contextlib.py", line 231, in __aexit__
  |     await self.gen.athrow(value)
  |   File "/home/hex/Documents/devops/voice-agent/.venv/lib/python3.12/site-packages/mcp/client/stdio/__init__.py", line 182, in stdio_client
  |     async with (
  |   File "/home/hex/Documents/devops/voice-agent/.venv/lib/python3.12/site-packages/anyio/_backends/_asyncio.py", line 783, in __aexit__
  |     raise BaseExceptionGroup(
  | ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Exception Group Traceback (most recent c

In [8]:
!npx -y @abhaybabbar/retellai-mcp-server

^C


pip install "mcp[cli]"

In [ ]:
# Base COde for MCP integration with Retell AI Voice Agents

import dotenv, os
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

dotenv.load_dotenv()  # Load environment variables from .env file


async def test_retell_mcp():
    # 1. Define the server parameters (matching your JSON config)
    server_params = StdioServerParameters(
        command="npx",
        args=["-y", "@abhaybabbar/retellai-mcp-server"],
        env={
            "RETELL_API_KEY": os.getenv("RETELL_API_KEY", "YOUR_RETELL_API_KEY_HERE"),
            "PATH": "/usr/local/bin:/usr/bin:/bin" # Ensure npx is in the path
        }
    )

    print("🚀 Connecting to Retell AI MCP Server...")

    # 2. Start the server and create a session
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # 3. List available tools (to see what's available)
            print("\n--- Available Tools ---")
            tools = await session.list_tools()
            for tool in tools.tools:
                print(f"Tool: {tool.name} - {tool.description}")

            # 4. Example: Access your Voice Agents
            # The tool name usually matches 'list_agents' or similar in this server
            print("\n--- Fetching Voice Agents ---")
            try:
                # We call the tool by name. Note: check the tool list output 
                # above to confirm the exact name (e.g., "list_agents")
                result = await session.call_tool("list_agents", arguments={})
                print("Result:", result.content[0].text)
            except Exception as e:
                print(f"❌ Error calling tool: {e}")

if __name__ == "__main__":
    asyncio.run(test_retell_mcp())

In [14]:
# 🔧 Alternative Approach: List Retell AI Capabilities via Direct API
import os
import requests
from dotenv import load_dotenv
import json

load_dotenv()

RETELL_API_KEY = os.getenv("RETELL_API_KEY")
BASE_URL = "https://api.retellai.com"

headers = {
    "Authorization": f"Bearer {RETELL_API_KEY}",
    "Content-Type": "application/json"
}

print("=" * 70)
print("🔍 RETELL AI - AVAILABLE API ENDPOINTS & TOOLS")
print("=" * 70)

# Define all available Retell AI endpoints
endpoints = {
    "Agents": [
        {"method": "GET", "path": "/list-agents", "description": "List all voice agents"},
        {"method": "POST", "path": "/create-agent", "description": "Create a new voice agent"},
        {"method": "GET", "path": "/get-agent/{agent_id}", "description": "Get agent details"},
        {"method": "PATCH", "path": "/update-agent/{agent_id}", "description": "Update agent configuration"},
        {"method": "DELETE", "path": "/delete-agent/{agent_id}", "description": "Delete an agent"},
    ],
    "Phone Calls": [
        {"method": "POST", "path": "/create-phone-call", "description": "Initiate outbound phone call"},
        {"method": "GET", "path": "/list-calls", "description": "List all phone calls"},
        {"method": "GET", "path": "/get-call/{call_id}", "description": "Get call details"},
    ],
    "Web Calls": [
        {"method": "POST", "path": "/create-web-call", "description": "Create web call access token"},
        {"method": "POST", "path": "/register-call", "description": "Register web call"},
    ],
    "Phone Numbers": [
        {"method": "GET", "path": "/list-phone-numbers", "description": "List purchased phone numbers"},
        {"method": "POST", "path": "/create-phone-number", "description": "Purchase new phone number"},
        {"method": "PATCH", "path": "/update-phone-number/{number}", "description": "Update phone number config"},
        {"method": "DELETE", "path": "/delete-phone-number/{number}", "description": "Release phone number"},
    ],
    "LLM & Prompts": [
        {"method": "GET", "path": "/list-llms", "description": "List available LLM models"},
        {"method": "POST", "path": "/create-custom-llm", "description": "Add custom LLM endpoint"},
    ],
    "Voice": [
        {"method": "GET", "path": "/list-voices", "description": "List available voices"},
    ],
    "Analytics": [
        {"method": "GET", "path": "/analytics", "description": "Get usage analytics"},
        {"method": "GET", "path": "/get-call-recording/{call_id}", "description": "Download call recording"},
    ]
}

# Display all available endpoints
for category, items in endpoints.items():
    print(f"\n📦 {category}")
    print("-" * 70)
    for item in items:
        print(f"   {item['method']:6} {item['path']:35} | {item['description']}")

print("\n" + "=" * 70)

# Test: List actual agents
print("\n🧪 TESTING: Fetching your actual voice agents...")
print("-" * 70)
try:
    response = requests.get(f"{BASE_URL}/list-agents", headers=headers)
    if response.status_code == 200:
        agents = response.json()
        print(f"✅ Found {len(agents)} agent(s):\n")
        for agent in agents:
            print(f"   🤖 Agent ID: {agent.get('agent_id', 'N/A')}")
            print(f"      Name: {agent.get('agent_name', 'Unnamed')}")
            print(f"      Voice: {agent.get('voice_id', 'N/A')}")
            print(f"      Language: {agent.get('language', 'N/A')}")
            print()
    else:
        print(f"❌ Error: {response.status_code} - {response.text}")
except Exception as e:
    print(f"❌ Error: {e}")

print("=" * 70)

🔍 RETELL AI - AVAILABLE API ENDPOINTS & TOOLS

📦 Agents
----------------------------------------------------------------------
   GET    /list-agents                        | List all voice agents
   POST   /create-agent                       | Create a new voice agent
   GET    /get-agent/{agent_id}               | Get agent details
   PATCH  /update-agent/{agent_id}            | Update agent configuration
   DELETE /delete-agent/{agent_id}            | Delete an agent

📦 Phone Calls
----------------------------------------------------------------------
   POST   /create-phone-call                  | Initiate outbound phone call
   GET    /list-calls                         | List all phone calls
   GET    /get-call/{call_id}                 | Get call details

📦 Web Calls
----------------------------------------------------------------------
   POST   /create-web-call                    | Create web call access token
   POST   /register-call                      | Register web call



In [ ]:
import dotenv, os
dotenv.load_dotenv()


os.getenv("RETELL_API_KEY", False)

In [8]:
server_params = StdioServerParameters(
        command="npx",
        args=["-y", "@abhaybabbar/retellai-mcp-server"],
        env={
            "RETELL_API_KEY": os.getenv("RETELL_API_KEY", "YOUR_RETELL_API_KEY_HERE"),
            "PATH": "/usr/local/bin:/usr/bin:/bin" # Ensure npx is in the path
        }
    )


RuntimeError: asyncio.run() cannot be called from a running event loop